# Przygotowanie
---
* otwóz plik Visum (`win32com.client`) 
* odczytaj warstwę POI
* oblicz macierze  kosztów dla `PT` (`jk`, i `ki`)
* optymalizuj typy danych
* złącz macierze do `jki`
* powtórz to samo dla `PRT`

> trzeba obliczać osobno dla PrT i PT, bo nie starcza RAMu

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import win32com.client
import os

params

In [ ]:
VISUM_PATH = os.path.join(os.getcwd(),"data//visum.ver")
VISUM_PATH = "E://Niedzielski//WBR.ver"
POI_CAT = 100
MTX_JRT = 15
MTX_PRT = 12

CZAS_PARKOWANIA = {"SPPN":6,
                   "POZA_SPPN":3}

In [3]:
Visum = win32com.client.Dispatch("Visum.Visum")  # uruchom Visum
Visum.LoadVersion(VISUM_PATH)  # zaladuj plik

In [4]:
POIs = pd.DataFrame(list(Visum.Net.POICategories.ItemByKey(POI_CAT).POIs.GetMultipleAttributes(["No", "ZoneID","Czas_Zone"])),
                    columns = ["POI", "ZoneID","Czas_Zone"])
POIs.ZoneID = POIs.ZoneID.astype(int)
POIs.POI = POIs.POI.astype(int)
POIs.Czas_Zone = POIs.Czas_Zone.astype('float32')/60

In [ ]:
cols = Visum.Net.Zones.GetMultiAttValues('No', False)
cols = [int(_[1]) for _ in cols]
mtx = Visum.Net.Matrices.ItemByKey(MTX_JRT)
vals = list(mtx.GetValuesFloat())
JRT = pd.DataFrame(vals, cols, cols)
mtx = Visum.Net.Matrices.ItemByKey(MTX_PRT)
vals = list(mtx.GetValuesFloat())
PRT = pd.DataFrame(vals, cols, cols)

In [6]:
JRT = JRT.stack().reset_index()
JRT.columns = ['Z',"Do","Czas_PuT"]
JRT.Czas_PuT = JRT.Czas_PuT.astype('float32')
PRT = PRT.stack().reset_index()
PRT.columns = ['Z',"Do","Czas_PrT"]
PRT.Czas_PrT = PRT.Czas_PrT.astype('float32')

In [7]:
del Visum

In [7]:
jk = pd.merge(JRT,POIs,how='outer',left_on="Do",right_on="ZoneID").dropna()

In [9]:
ki = pd.merge(POIs,JRT,how='outer',left_on="ZoneID",right_on="Z").dropna()

In [10]:
jki_JRT = pd.merge(jk,ki,on='POI')

In [22]:
jki_JRT['Czas_PuT']=jki_JRT.Czas_PuT_x + jki_JRT.Czas_PuT_y + 2*jki_JRT.Czas_Zone_x  

In [23]:
del jki_JRT['Czas_PuT_x']
del jki_JRT['Czas_PuT_y']
del jki_JRT['Do_x']
del jki_JRT['ZoneID_x']
del jki_JRT['ZoneID_y']
del jki_JRT['Z_y']
del jki_JRT['Czas_Zone_x']
del jki_JRT['Czas_Zone_y']

In [24]:
jki_JRT.columns = ['Z_Rejon','POI','Do_Rejon','Czas_PuT']

In [ ]:
jki_JRT.to_csv("data//From_Via_To_JRT.csv", chunksize=20000000)

In [16]:
del jki_JRT

In [207]:
jk = pd.merge(PRT,POIs,how='outer',left_on="Do",right_on="ZoneID").dropna()
ki = pd.merge(POIs,PRT,how='outer',left_on="ZoneID",right_on="Z").dropna()
jki_PRT = pd.merge(jk,ki,on='POI')

In [208]:
jki_PRT['Czas_PrT']=jki_PRT.Czas_PrT_x + jki_PRT.Czas_PrT_y + 2*jki_PRT.Czas_Zone_x + 2*CZAS_PARKOWANIA['SPPN'] 

jki_PRT.columns = ['Z_Rejon','POI','Do_Rejon','Czas_PrT']

In [ ]:
del jki_PRT['Czas_PrT_x']
del jki_PRT['Czas_PrT_y']
del jki_JRT['Do_x']
del jki_PRT['ZoneID_x']
del jki_PRT['ZoneID_y']
del jki_PRT['Z_y']
del jki_PRT['Czas_Zone_x']
del jki_PRT['Czas_Zone_y']

In [ ]:
jki_PRT.to_csv("data//From_Via_To_PRT.csv", chunksize=20000000)

In [211]:
jki = pd.merge(jki_PRT,jki_JRT,on=['Z_Rejon','POI','Do_Rejon'])
jki.to_csv("data//From_Via_To_.csv")

In [210]:
jki.head(1)

,Z_Rejon,POI,Do_Rejon,Czas_PrT,Czas_PuT
0,1,15.00,1,876.23,156.23
